# Machine Learning

## Learning Objectives
- Understand the basic concepts of supervised and unsupervised machine learning, how this differs from modeling for interpretation (which they are most likely more familiar with), and how it can be used for policy applications.
- Use ML packages in Python to bring in individual-level data combined across multiple sources; determine and generate appropriate features, outcome variables, evaluation methods and training/test splits; identify a best model and conduct error analysis and provide interpretation within context.




In [1]:
import numpy
import pandas
import statsmodels
import sklearn

### The Machine Learning Process

- **Understand the problem and goal. This sounds obvious but is often nontrivial.** Problems typically start as vague 
descriptions of a goal - improving health outcomes, increasing graduation rates, understanding the effect of a 
variable *X* on an outcome *Y*, etc. It is really important to work with people who understand the domain being
studied to dig deeper and define the problem more concretely. What is the analytical formulation of the metric 
that you are trying to optimize?
- **Formulate it as a machine learning problem.** Is it a classification problem or a regression problem? Is the 
goal to build a model that generates a ranked list prioritized by risk, or is it to detect anomalies as new data 
come in? Knowing what kinds of tasks machine learning can solve will allow you to map the problem you are working on
to one or more machine learning settings and give you access to a suite of methods.
- **Data exploration and preparation.** Next, you need to carefully explore the data you have. What additional data
do you need or have access to? What variable will you use to match records for integrating different data sources?
What variables exist in the data set? Are they continuous or categorical? What about missing values? Can you use the 
variables in their original form, or do you need to alter them in some way?
- **Feature engineering.** In machine learning language, what you might know as independent variables or predictors 
or factors or covariates are called "features." Creating good features is probably the most important step in the 
machine learning process. This involves doing transformations, creating interaction terms, or aggregating over data
points or over time and space.
- **Method selection.** Having formulated the problem and created your features, you now have a suite of methods to
choose from. It would be great if there were a single method that always worked best for a specific type of problem, 
but that would make things too easy. Typically, in machine learning, you take a collection 
- **Evaluation.** As you build a large number of possible models, you need a way to select the model that is the 
best. This part of the chapter will cover the validation methodology to first validate models on historical data
as well as discuss a variety of evaluation metrics. The next step is to validate using a field trial or experiment.
- **Deployment.** Once you have selected the best model and validated it using historical data as well as a field
trial, you are ready to put the model into practice. You still have to keep in mind that new data will be coming in,
and the model might change over time.
